# AutoMLを活用したモデル構築

AutoMLを活用したモデルトレーニングおよびMLflowトラッキングを行います

**注意**
ノートブックから時系列予測のAutoMLを行う際、フォルダ名、ノートブック名にマルチバイト文字(日本語)を含めないでください。エラーになります。

**要件**
クラスタ Runtime 15.4 ML LTS以上を使用してください。

<!-- %md
### Model Building Using AutoML
We will perform model training using AutoML and track the results with MLflow. -->


<img src='https://github.com/komae5519pv/komae_dbdemos/blob/main/fine_grain_forecast_20241013/Customized_e2e_demand_forecasting/_image_for_notebook/model_train.png?raw=true' width='1200'/>

## Step1. AutoMLトレーニング

以下のコマンドはAutoMLのランを起動します。予測すべきカラムを`target_col`引数と時間のカラムを指定する必要があります。ランが完了すると、トレーニングコードを検証するためにベストなトライアルのノートブックへのリンクにアクセスすることができます。

このサンプルでは以下の指定も行っています。
- `horizon=90`, AutoMLが未来の90日を予測するように指定 
- `frequency="d"`, 日毎の予測を行うことを指定 
- `primary_metric="mdape"`, トレーニングの際に最適化すべきメトリックを指定

In [0]:
%run ./00_config

### トレーニングデータの準備

日毎の店舗・アイテム・日付ごとの売上を含むデータセットです。  
ここでのゴールは、向こう90日の店舗・アイテム・日付ごとの売上の予測を行うことです。

<!-- %md  
### Prepare the Training Data

The dataset contains daily sales for each store, item, and date.  
The goal here is to forecast the sales for each store, item, and date for the next 90 days. -->

In [0]:
vm = 1
item = 1

query = f'''
  SELECT
    vm,
    item,
    CAST(ds as date) as ds,
    SUM(y) as y
  FROM silver_train
  WHERE vm = {vm} AND item = {item}  -- 単一店舗x単一商品
  GROUP BY vm, item, ds
  ORDER BY vm, item, ds
  '''

train_df = (
  spark
    .sql( query )
    .repartition(sc.defaultParallelism, ['vm', 'item'])
    .dropna()
  ).cache()

display(train_df.orderBy(['ds','vm','item']))

In [0]:
# vm = 1
# item = 1

# query = f'''
#   SELECT
#     vending_machine_id as vm,
#     item_id as item,
#     CAST(order_date as date) as ds,
#     SUM(sales_quantity) as y
#   FROM bronze_train
#   WHERE vending_machine_id = {vm} AND item_id = {item}  -- 単一店舗x単一商品
#   GROUP BY vm, item, ds
#   ORDER BY vm, item, ds
#   '''

# train_df = (
#   spark
#     .sql( query )
#     .repartition(sc.defaultParallelism, ['vm', 'item'])
#     .dropna()
#   ).cache()

# display(train_df.orderBy(['ds','vm','item']))

### AutoMLを活用したトレーニング実行

<!-- %md  
## Step 2: Execute Training Using AutoML -->

In [0]:
from databricks import automl
import logging

# Prophetの情報レベルのメッセージを無効化
logging.getLogger("py4j").setLevel(logging.WARNING)

In [0]:
from datetime import datetime

# エクスペリメントのパスと名前を設定
xp_path = f"/Shared/{MY_CATALOG}_demand_forecast_automl"
xp_name = f"demand_forecast_store_{vm}_item_{item}_{datetime.now().strftime('%Y-%m-%d_%H:%M:%S')}"

# AutoMLを使用してモデルをトレーニング
automl_run = automl.forecast(
    experiment_name=xp_name,    # 実験名
    experiment_dir=xp_path,     # 実験ディレクトリ
    dataset=train_df,           # データセット
    target_col="y",             # 予測対象の列
    time_col="ds",              # 時間を表す列
    frequency="d",              # データの頻度（日次）
    horizon=90,                 # 90日先まで予測
    timeout_minutes=10,         # タイムアウト時間（分）
    primary_metric="mdape",     # 評価メトリクス（MDAPE）
    country_code="US",          # 祝日の対象国（米国）
)

### 次のステップ

* 上のノートブックとエクスペリメントを探索
* ベストなトライアルのノートブックのメトリックが好適であれば、次のセルに進むことができます。
* ベストトライアルによるモデルを改善したいのであれば、以下をトライします。
  * ベストトライアルのノートブックに移動し、クローンします。
  * モデルを改善するためにノートブックに必要な修正を加えます。
  * モデルに満足したら、トレーニングされたモデルが記録されているアーティファクトのURIをメモします。そのURIを次のセルの`model_uri`に指定します。

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import iam

# 全ユーザー(グループ名: account users)にエクスペリメントの権限を設定
w = WorkspaceClient()
try:
    status = w.workspace.get_status(f"{xp_path}/{xp_name}")
    w.permissions.set("experiments", request_object_id=status.object_id, access_control_list=[
        iam.AccessControlRequest(group_name="account users", permission_level=iam.PermissionLevel.CAN_MANAGE)
    ])
    print(f"Experiment on {xp_path}/{xp_name} was set public")
except Exception as e:
    print(f"Error setting up shared experiment {xp_path}/{xp_name} permission: {e}")

# # 全ユーザーがdbdemos共有実験にアクセスできるようにする
# DBDemos.set_experiment_permission(f"{xp_path}/{xp_name}")

## Step 2: MLflowを用いてベストモデルをロード

MLflowの`mlflow.pyfunc`を用いて、AutoMLの`trial_id`を用いてモデルを容易にPythonにインポートすることができます。

In [0]:
import mlflow.pyfunc
from mlflow.tracking import MlflowClient

run_id = MlflowClient()
trial_id = automl_run.best_trial.mlflow_run_id

model_uri = "runs:/{run_id}/model".format(run_id=trial_id)
pyfunc_model = mlflow.pyfunc.load_model(model_uri)

## Step 3: ロードしたモデルで予測する

予測を行うために`predict_timeseries`のモデルメソッドを呼び出します。詳細は[Prophet documentation](https://facebook.github.io/prophet/docs/quick_start.html#python-api)をご覧ください。

In [0]:
# 予測を生成（過去データも含めて未来90日間）
forecasts = pyfunc_model._model_impl.python_model.predict_timeseries(include_history=True)

display(forecasts)

Databricks visualization. Run in Databricks to view.

## Step 4: 予測の変化点とトレンドをプロット

以下のプロットでは、太い黒線は時系列データセットを示しており、青い線がモデルによる予測値を示しています。

In [0]:
from pyspark.sql import functions as F

df_true = train_df.groupBy("ds").agg(F.avg("y").alias("y")).toPandas()
# display(df_true)

In [0]:
import matplotlib.pyplot as plt
import pandas as pd

# 日付['ds'] を datetime 型に変換
df_true['ds'] = pd.to_datetime(df_true['ds'], errors='coerce')
forecasts['ds'] = pd.to_datetime(forecasts['ds'], errors='coerce')

# プロット
fig = plt.figure(facecolor='w', figsize=(10, 6))
ax = fig.add_subplot(111)
forecasts = pyfunc_model._model_impl.python_model.predict_timeseries(include_history=True)
fcst_t = forecasts['ds'].dt.to_pydatetime()
ax.plot(df_true['ds'].dt.to_pydatetime(), df_true['y'], 'k.', label='Observed data points')
ax.plot(fcst_t, forecasts['yhat'], ls='-', c='#0072B2', label='Forecasts')
ax.fill_between(fcst_t, forecasts['yhat_lower'], forecasts['yhat_upper'],
                color='#0072B2', alpha=0.2, label='Uncertainty interval')
ax.legend()
plt.show()

## Step 5: 予測結果テーブルの作成

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DateType

# 必要な列のみを train_df から取得（vm, item, ds, y）
train_df_selected = train_df.select("vm", "item", "ds", "y")

# forecasts を Spark DataFrame に変換
forecast_spark = spark.createDataFrame(forecasts)

# 予測結果に vm と item の情報を結合
forecast_spark = forecast_spark.join(train_df_selected, on='ds', how='left')

# ds列を正しい日付形式に変換
forecast_spark = forecast_spark.withColumn('ds', F.col('ds').cast(DateType()))

# 新しい予測結果を一時テーブルとして保存
forecast_spark.createOrReplaceTempView('new_forecasts_automl')

display(forecast_spark)

In [0]:
%sql

-- Deltaテーブルにマージして保存
CREATE TABLE IF NOT EXISTS silver_forecasts_automl (
  order_date DATE,
  vending_machine_id INTEGER,
  item_id INTEGER,
  actual_sales_quantity FLOAT,
  forecast_sales_quantity FLOAT,
  forecast_sales_quantity_upper FLOAT,
  forecast_sales_quantity_lower FLOAT,
  sales_inference_date DATE
  )
USING DELTA
PARTITIONED BY (order_date);

MERGE INTO silver_forecasts_automl f
USING new_forecasts_automl n
ON f.order_date = n.ds AND f.vending_machine_id = n.vm AND f.item_id = n.item
WHEN MATCHED THEN UPDATE SET
  f.order_date = n.ds,
  f.vending_machine_id = n.vm,
  f.item_id = n.item,
  f.actual_sales_quantity = n.y,
  f.forecast_sales_quantity = n.yhat,
  f.forecast_sales_quantity_upper = n.yhat_upper,
  f.forecast_sales_quantity_lower = n.yhat_lower,
  f.sales_inference_date = current_date()
WHEN NOT MATCHED THEN INSERT (
  order_date,
  vending_machine_id,
  item_id,
  actual_sales_quantity,
  forecast_sales_quantity,
  forecast_sales_quantity_upper,
  forecast_sales_quantity_lower,
  sales_inference_date)
VALUES (
  n.ds,
  n.vm,
  n.item,
  n.y,
  n.yhat,
  n.yhat_upper,
  n.yhat_lower,
  current_date());

In [0]:
# テーブル名
table_name = f'{MY_CATALOG}.{MY_SCHEMA}.silver_forecasts_automl'

# テーブルコメント
comment = """
`silver_forecasts_automl`テーブルは、自動販売機の需要予測結果データを管理します。AutoMLで構築したモデルを使用しています。
"""
spark.sql(f'COMMENT ON TABLE {table_name} IS "{comment}"')

# カラムコメント
column_comments = {
    "order_date": "受注日（主キー、外部キー）、YYYY-MM-DDフォーマット",
    "vending_machine_id": "自動販売機ID（主キー、外部キー）、例: 10",
    "item_id": "商品ID（主キー、外部キー）、例: 10",
    "actual_sales_quantity": "実績販売数、例: 50",
    "forecast_sales_quantity": "予測販売数、例: 50",
    "forecast_sales_quantity_upper": "予測販売数（上限）、例: 60",
    "forecast_sales_quantity_lower": "予測販売数（下限）、例: 40",
    "sales_inference_date": "販売数予測日、YYYY-MM-DDフォーマット"
}

for column, comment in column_comments.items():
    # シングルクォートをエスケープ
    escaped_comment = comment.replace("'", "\\'")
    sql_query = f"ALTER TABLE {table_name} ALTER COLUMN {column} COMMENT '{escaped_comment}'"
    spark.sql(sql_query)

これで、各店舗・アイテムの組み合わせごとの予測を作成し、基本的な評価指標を生成しました。この予測データを確認するには、シンプルなクエリを発行できます（ここでは、商品1を対象に店舗1から3までに限定しています）。

<!-- %md
We now have constructed a forecast for each store-item combination and generated basic evaluation metrics for each.  To see this forecast data, we can issue a simple query (limited here to product 1 across stores 1 through 3): -->

In [0]:
%sql

SELECT
  vending_machine_id,
  order_date,
  forecast_sales_quantity,
  forecast_sales_quantity_upper,
  forecast_sales_quantity_lower
FROM silver_forecasts_automl a
WHERE item_id = 1 AND
      -- vending_machine_id IN (1, 2, 3) AND
      vending_machine_id IN (1) AND
      order_date >= '2013-01-01' AND
      sales_inference_date=current_date()
ORDER BY vending_machine_id

Databricks visualization. Run in Databricks to view.

## Step 3: ベストモデルのUnity Catalogに登録
モデルの準備が整いました。AutoMLの実行で生成されたノートブックを確認し、必要に応じてカスタマイズすることができます。  
このデモでは、モデルが準備できていると仮定し、Model Registryに本番環境としてデプロイします。

ベストモデルをProphetWrapperでラップし、カスタム予測ロジックを追加し、yhat, yhat_upper, yhat_lowerなどの追加情報を確実に出力できるようにします。  
その上でカスタマイズしたモデル（ProphetWrapper）をMLflowにログします。 このモデルをUnity Catalog（UC）に登録します。
<!-- %md  
## Deploy the Model to Production
The model is now ready. You can review the notebook generated by the AutoML run and customize it as needed.
In this demo, we assume that the model is ready and will deploy it as a production model in the Model Registry. -->

In [0]:
from mlflow import MlflowClient
from databricks.sdk import WorkspaceClient
import databricks.sdk.service.catalog as c

# Databricks Unity Catalogを使用してモデルを保存します
mlflow.set_registry_uri('databricks-uc')
client = MlflowClient()

# カタログにモデルを追加
latest_model = mlflow.register_model(f'runs:/{automl_run.best_trial.mlflow_run_id}/model', MODEL_NAME_AUTOML)

# UCエイリアスを使用してプロダクション対応としてフラグを立てる
client.set_registered_model_alias(name=f"{MODEL_NAME_AUTOML}", alias="prod", version=latest_model.version)

# WorkspaceClientのインスタンスを作成
sdk_client = WorkspaceClient()

# 全ユーザー(グループ名: account users)にモデルの権限を設定
sdk_client.grants.update(c.SecurableType.FUNCTION, f"{MY_CATALOG}.{MY_SCHEMA}.{MODEL_NAME_AUTOML}", 
                         changes=[c.PermissionsChange(add=[c.Privilege["ALL_PRIVILEGES"]], principal="account users")])